In [1]:
import pandas as pd
import numpy as np
# The never-made-up COL-ATL game is not here:
# df.loc[(df['date'] == 20210916)]


def load_game_log(game_log_path):
    retro_df_columns=['date', 'doubleheader_index','weekday', 'visitor', 'visitor_league', 'visitor_game_num', 'home', 'home_league', 'home_game_num', 'visitor_score', 'home_score', 'length_outs', 'day_night', 'completion']
    df = pd.read_csv(game_log_path, header=None, names=retro_df_columns, usecols=range(len(retro_df_columns)))
    df['date'] = df['date'].astype(str)
    df = df.fillna(value={'completion': 'I hate Pandas'})
    df['completion'] = df['completion'].astype(str)
    df['completion_date'] = df['date']
    df.loc[(df['completion'] != "I hate Pandas"), 'completion_date'] = df['completion'].str.split(',').str[0]
    df['completion_date'] = df['completion_date'].astype(str)
    return df

df = load_game_log('./data/GL2021.txt')
df.loc[(df['date'] != df['completion_date'])]
# df.loc[(df['visitor'] == 'PIT') | (df['home'] == 'PIT')].shape


,date,doubleheader_index,weekday,visitor,visitor_league,visitor_game_num,home,home_league,home_game_num,visitor_score,home_score,length_outs,day_night,completion,completion_date
123,20210411,0,Sun,MIA,NL,9,NYN,NL,6,5,6,53,D,"20210831,NYC20,0,0,1",20210831
721,20210526,0,Wed,CIN,NL,47,WAS,NL,45,3,5,51,D,"20210527,,0,3,21",20210527
1367,20210717,0,Sat,MIA,NL,93,PHI,NL,91,2,4,58,D,"20210718,PHI13,2,2,54",20210718
1370,20210717,0,Sat,SDN,NL,95,WAS,NL,91,10,4,54,D,"20210718,WAS11,8,4,33",20210718
1422,20210721,2,Wed,SDN,NL,99,ATL,NL,95,6,5,42,N,"20210924,SAN02,5,4,27",20210924
1687,20210810,0,Tue,WAS,NL,113,NYN,NL,113,7,8,51,D,"20210811,NYC20,3,1,6",20210811


In [2]:
# adapted from sdvinay
def compute_standings(game_log):
    gms_played = game_log.copy()
    gms_played = gms_played[gms_played['visitor_score'] != gms_played['home_score']]
    margins = gms_played['visitor_score']-gms_played['home_score']
    winners = pd.Series(np.where(margins>0, gms_played['visitor'], gms_played['home']))
    losers  = pd.Series(np.where(margins<0, gms_played['visitor'], gms_played['home']))
    standings = pd.concat([winners.value_counts().rename('W'), losers.value_counts().rename('L')], axis=1)
    return standings.fillna(0)

compute_standings(df)# .sub(compute_standings(df.loc[(df['completion_date'] == '20211003')]), fill_value=0)

,W,L
SFN,107,55
LAN,106,56
TBA,100,62
HOU,95,67
MIL,95,67
CHA,93,69
NYA,92,70
BOS,92,70
TOR,91,71
SEA,90,72


In [3]:
standings = compute_standings(df)
# standings['div'] = divisions['div']
# standings['lg'] = divisions['lg']
standings

,W,L
SFN,107,55
LAN,106,56
TBA,100,62
HOU,95,67
MIL,95,67
CHA,93,69
NYA,92,70
BOS,92,70
TOR,91,71
SEA,90,72


In [4]:
# max_wins
162 -  standings['L']


SFN    107
LAN    106
TBA    100
HOU     95
MIL     95
CHA     93
NYA     92
BOS     92
TOR     91
SEA     90
SLN     90
ATL     89
OAK     86
CIN     83
PHI     82
CLE     80
SDN     79
DET     77
NYN     77
ANA     77
KCA     74
COL     75
MIN     73
CHN     71
MIA     67
WAS     65
PIT     61
TEX     60
ARI     52
BAL     52
Name: L, dtype: int64

In [5]:
def load_nicknames(nicknames_file):
    retro_nickname_columns=['current_id', 'contemporary_id', 'league', 'division', 'location', 'nickname', 'alt_nickname', 'start_date', 'end_date']
    df = pd.read_csv(nicknames_file, header=None, names=retro_nickname_columns, usecols=range(len(retro_nickname_columns)))
    return df

def divisions_for_year(nicknames_from_retrosheet, year):
    df = nicknames_from_retrosheet.copy()
    # df['start_date'] = df['start_date'].astype(str)
    df['start_year'] = pd.DatetimeIndex(df['start_date']).year
    df['end_year'] = pd.DatetimeIndex(df['end_date']).year
    df = df.loc[(df['start_year'] <= year) & ((df['end_year'].isna()) | (df['end_year'] >= year))] # & (df['end_year'] => year)]
    df['div'] = df[['league', 'division']].fillna('').sum(axis=1)
    
    return df.set_index('contemporary_id')[['league', 'div']].rename(columns={'league': 'lg'})

NICKNAMES = load_nicknames('data/CurrentNames.csv')
DIVISIONS = divisions_for_year(NICKNAMES, 2021)


In [6]:
# standings.loc[(standings['div'] == 'NE')]['W'].max()

def division_contenders(standings_immutable, divisions, season_length):
    df = standings_immutable.copy()
#    print(f'division_contenders was called with columns {df.columns}')
    df['div'] = divisions['div']
    max_wins = season_length - df['L']
    division_contenders = set()
    for index, row in df.iterrows():
        most_wins_in_division = df.loc[(df['div'] == row["div"])]['W'].max()
        if max_wins[index] >= most_wins_in_division:
            division_contenders.add(index)
    return division_contenders
print(f'Still in contention: {division_contenders(standings, DIVISIONS, 162)}')

Still in contention: {'CHA', 'ATL', 'TBA', 'SFN', 'HOU', 'MIL'}


In [7]:
from datetime import datetime, timedelta

def retro_to_datetime(retro_str):
    return datetime.strptime(retro_str, '%Y%m%d')

def datetime_to_retro(dt):
    return datetime.strftime(dt, '%Y%m%d')

def show_dumb_elimination_output(df, divisions):
    max_date = retro_to_datetime(df['completion_date'].max())
    min_date = retro_to_datetime(df['completion_date'].min())

    current_date = max_date
    contenders = set()
    old_contenders = None
    while current_date > min_date and len(contenders) < 30:
      date_str = datetime_to_retro(current_date)
      current_standings = compute_standings(df.loc[df['completion_date'] <= date_str])
      # MOVE THIS SHIT INTO SOMETHING EFFICIENT
      current_standings['div'] = divisions['div']
      current_standings['lg'] = divisions['lg']
      current_standings['max_wins'] = 162 - current_standings['L']
      contenders = division_contenders(current_standings, divisions, 162)
      if old_contenders:
        new_contenders = contenders.difference(old_contenders)
        if new_contenders:
          print(f'Teams eliminated from their division titles on {datetime_to_retro(tomorrow)}: {new_contenders}')
      tomorrow = current_date
      old_contenders = contenders.copy()
      current_date = current_date - timedelta(days=1)

show_dumb_elimination_output(df, DIVISIONS)

Teams eliminated from their division titles on 20211003: {'LAN'}
Teams eliminated from their division titles on 20210930: {'SEA', 'PHI'}
Teams eliminated from their division titles on 20210927: {'OAK'}
Teams eliminated from their division titles on 20210926: {'SLN'}
Teams eliminated from their division titles on 20210925: {'NYA', 'NYN', 'BOS'}
Teams eliminated from their division titles on 20210924: {'TOR'}
Teams eliminated from their division titles on 20210923: {'CLE'}
Teams eliminated from their division titles on 20210919: {'DET', 'MIA'}
Teams eliminated from their division titles on 20210918: {'CIN', 'ANA'}
Teams eliminated from their division titles on 20210917: {'WAS'}
Teams eliminated from their division titles on 20210916: {'KCA'}
Teams eliminated from their division titles on 20210913: {'SDN', 'MIN'}
Teams eliminated from their division titles on 20210910: {'CHN'}
Teams eliminated from their division titles on 20210906: {'COL'}
Teams eliminated from their division titles on 2

In [8]:
def load_schedule(filename):
    schedule_columns=['date', 'doubleheader_index','weekday', 'visitor', 'visitor_league', 'visitor_game_num', 'home', 'home_league', 'home_game_num', 'day_night', 'completion', 'makeup_date']
    df = pd.read_csv(filename, header=None, names=schedule_columns, usecols=range(len(schedule_columns)))
    df['completion'] = df['completion'].astype(str)
    df['makeup_date'] = df['makeup_date'].astype(str)
    return df.dropna(subset=['home'])

schedule_path = './data/2021SKED.TXT'
SCHEDULE = load_schedule(schedule_path)

def find_unplayed_games(schedule):
    return schedule.loc[schedule["makeup_date"].str.startswith("not", na=False)]

find_unplayed_games(load_schedule(schedule_path))

,date,doubleheader_index,weekday,visitor,visitor_league,visitor_game_num,home,home_league,home_game_num,day_night,completion,makeup_date


In [9]:
def logged_games_after_date(df, date_str):
    return df.loc[df['completion_date'] > date_str]

def all_matchups_after_date(df, schedule, date_str):
    logged_games = logged_games_after_date(df, date_str)[['date', 'doubleheader_index', 'visitor','home']]
    unplayed_games = find_unplayed_games(schedule)[['date', 'doubleheader_index', 'visitor','home']]
    all_games = pd.concat([logged_games, unplayed_games], ignore_index=True)
    alpha_pairs = pd.DataFrame(np.where(all_games['visitor'] < all_games['home'],
                                     (all_games['visitor'], all_games['home']),
                                     (all_games['home'], all_games['visitor'])))
    return alpha_pairs.T.rename(columns={0: 'alpha1', 1: 'alpha2'})

all_matchups_after_date(df, load_schedule(schedule_path), '20211001').groupby(['alpha1', 'alpha2'], as_index=False).size()


,alpha1,alpha2,size
0,ANA,SEA,2
1,ARI,COL,2
2,ATL,NYN,2
3,BAL,TOR,2
4,BOS,WAS,2
5,CHA,DET,2
6,CHN,SLN,2
7,CIN,PIT,2
8,CLE,TEX,2
9,HOU,OAK,2


In [10]:

def divisional_threats(standings_immutable, divisions, season_length, team):
    # We only care about teams with more max_wins than us.
    df = standings_immutable.copy()
    my_division = divisions.loc[team]['div']
    df = df.merge(divisions, left_index=True, right_index=True).loc[divisions['div'] == my_division]   
    max_wins = season_length - df['L']
    df = df.loc[(df.index != team) & (max_wins[df.index] >= max_wins[team])]
    return max_wins[team] - df['W']
#    return df.loc[(df.index != team) & (divisions.loc[df.index] == my_division) & (max_wins[df.index] >= max_wins[team])]
    # return df.merge(divisions, left_index=True, right_index=True(df.index != team) & (max_wins[df.index] >= max_wins[team])]
#standings.loc['NYN']['div']
divisional_threats(standings, DIVISIONS, 162, 'NYN')
#standings.merge(divisions, left_index=True, right_index=True).loc[divisions['div'] == 'NE']

ATL   -11
PHI    -5
Name: W, dtype: int64

In [11]:
orioles_test = compute_standings(df.loc[(df['completion_date'] <= '20210823')])
threats1 = divisional_threats(orioles_test, DIVISIONS, 162, 'BAL')
threats1
# why_cant_i_do_this_in_one_line.merge(divisions, left_index=True, right_index=True)
# orioles_test = why_cant_i_do# _this_in_one_line.loc[divisions[why_cant_i_do_this_in_one_line.index] == 'AE']

TBA     0
NYA     4
BOS     6
TOR    12
Name: W, dtype: int64

In [12]:
def games_between_rivals_after_date(df, schedule, divisions, date_str, team):
    remaining = all_matchups_after_date(df, schedule, date_str).groupby(['alpha1', 'alpha2'], as_index=False).size()
    standings = compute_standings(df.loc[(df['completion_date'] <= date_str)])
    season_length = get_season_length(schedule) # why god why
    threats = divisional_threats(standings, divisions, season_length, team)
    return remaining.loc[(remaining['alpha1'].isin(threats.index)) & (remaining['alpha2'].isin(threats.index))]



In [13]:
# I have to sort games_among_threats in decreasing order of team quality
# Maybe just replace the team names with ints?


In [14]:
def get_season_length(schedule):
    unique_game_counts = pd.unique(pd.concat([schedule['home'], schedule['visitor']], axis = 0).value_counts())
    assert(unique_game_counts.shape == (1,))
    return unique_game_counts[0]

get_season_length(SCHEDULE)

162

In [15]:
# sort the output of division_threats
def sort_rivals(matchups, threats):
    # print(matchups.columns, threats.shape)
    df = matchups.copy()
    merge1 = df.merge(threats, left_on=['alpha1'], right_index=True)
    # print(f'now merge1 is: {merge1}')
    merge2 = merge1.merge(threats, left_on=['alpha2'], right_index=True)
    # print(f'now merge2 is: {merge2}')
    merge2['betterT'] = np.where(merge2['W_x'] >= merge2['W_y'], merge2['alpha2'], merge2['alpha1'])
    merge2['worseT'] = np.where(merge2['W_x'] >= merge2['W_y'], merge2['alpha1'], merge2['alpha2'])
    merge2['lesserW'] = merge2[['W_x','W_y']].min(axis=1)
    merge2['greaterW'] = merge2[['W_x','W_y']].max(axis=1)
    return merge2.sort_values(['lesserW', 'greaterW'], ascending=True)[['betterT', 'worseT', 'size']]
    
matchups1 = games_between_rivals_after_date(df, load_schedule(schedule_path), DIVISIONS, '20210823', 'BAL')
threats1 = divisional_threats(compute_standings(df.loc[(df['completion_date'] <= '20210823')]), DIVISIONS, 162, 'BAL')
sorted_rivals1 = sort_rivals(matchups1, threats1)
sorted_rivals1['size']

112    3
40     7
129    6
37     3
114    7
Name: size, dtype: int64

In [16]:
def portion_out_wins(sorted_rivals, threats):
    for _, row in sorted_rivals.iterrows():
        games = row['size']
        betterT = row['betterT']
        worseT = row['worseT']
        threat1, threat2 = threats[betterT], threats[worseT]
        print(f'We can afford to let {betterT} win {threat1} more games, and {worseT} {threat2}')
        if games > threat1 + threat2:
            # No matter what, these games will lead to too many wins for one team.
            print(f'But there are {games} games left between {betterT} and {worseT}')
            return False
        # We try to give as many wins to the best team as possible first. Then distribute them
        # downward. Not sure about this.
        # In the Baltimore example, Tampa Bay cannot win another game. So wins_for_t1 is going to be
        # zero. We give all three of the TB-NY series to NY.
        wins_for_t1 = min(threat1, games)
        wins_for_t2 = games - wins_for_t1
        print(f'Let\'s say that {betterT} wins {wins_for_t1} of their remaining {games} with {worseT}')
        threats[row['betterT']] -= wins_for_t1
        threats[row['worseT']] -= wins_for_t2
    return True

matchups1 = games_between_rivals_after_date(df, load_schedule(schedule_path), DIVISIONS, '20210822', 'PHI')
threats1 = divisional_threats(compute_standings(df.loc[(df['completion_date'] <= '20210822')]), DIVISIONS, 162, 'PHI')
sorted_rivals1 = sort_rivals(matchups1, threats1)
threats1
# portion_out_wins(sorted_rivals1, threats1)                                                           

ATL    33
Name: W, dtype: int64

In [49]:
def is_division_contender_with_rivalries(game_log, schedule, divisions, date_str, team):
    matchups = games_between_rivals_after_date(game_log, schedule, divisions, date_str, team)
    season_length = get_season_length(schedule) # why god why
    threats = divisional_threats(compute_standings(game_log.loc[(game_log['completion_date'] <= date_str)]), divisions, season_length, team)
    sorted_rivals = sort_rivals(matchups, threats)
    return all_subset_sums(sorted_rivals, threats)

is_division_contender_with_rivalries(df, SCHEDULE, DIVISIONS,  '20210822', 'BAL')

KeyError: 'alpha1'

In [18]:
def show_slightly_less_dumb_elimination_output(df, schedule, divisions):
    max_date = retro_to_datetime(df['completion_date'].max())
    min_date = retro_to_datetime(df['completion_date'].min())

    current_date = max_date
    contenders = set()
    old_contenders = None
    while current_date > min_date and len(contenders) < 30:
        date_str = datetime_to_retro(current_date)
        current_standings = compute_standings(df.loc[df['completion_date'] <= date_str])
        # MOVE THIS SHIT INTO SOMETHING EFFICIENT
        current_standings['div'] = divisions['div']
        current_standings['lg'] = divisions['lg']
        current_standings['max_wins'] = 162 - current_standings['L']
    
        contenders = division_contenders(current_standings, divisions, 162)
        for supposed_contender in contenders.copy():
            # If you're in contention tomorrow, you're in contention today, so I am not going to
            # waste CPU time on you.
            if old_contenders and supposed_contender not in old_contenders:
                if not is_division_contender_with_rivalries(df, schedule, divisions, date_str, supposed_contender):
                    print(f'It looked like {supposed_contender} was in contention after {date_str} but the remaining intra-division games ruled them out.')
                    contenders.remove(supposed_contender)
        new_contenders = set()
        if old_contenders:
            new_contenders = contenders.difference(old_contenders)
        if new_contenders:
            print(f'Teams eliminated from their division titles on {datetime_to_retro(tomorrow)}: {new_contenders}')
        tomorrow = current_date
        old_contenders = contenders.copy()
        current_date = current_date - timedelta(days=1)

show_slightly_less_dumb_elimination_output(df, SCHEDULE, DIVISIONS)

Teams eliminated from their division titles on 20211003: {'LAN'}
Teams eliminated from their division titles on 20210930: {'SEA', 'PHI'}
Teams eliminated from their division titles on 20210927: {'OAK'}
Teams eliminated from their division titles on 20210926: {'SLN'}
We can afford to let ATL win 1 more games, and PHI 2
Let's say that ATL wins 1 of their remaining 3 with PHI
Teams eliminated from their division titles on 20210925: {'NYA', 'NYN', 'BOS'}
We can afford to let TBA win 0 more games, and NYA 8
Let's say that TBA wins 0 of their remaining 3 with NYA
We can afford to let BOS win 6 more games, and NYA 5
Let's say that BOS wins 3 of their remaining 3 with NYA
Teams eliminated from their division titles on 20210924: {'TOR'}
Teams eliminated from their division titles on 20210923: {'CLE'}
We can afford to let PHI win 0 more games, and ATL 0
But there are 3 games left between PHI and ATL
It looked like MIA was in contention after 20210918 but the remaining intra-division games ruled 

We can afford to let SFN win 0 more games, and LAN 1
But there are 3 games left between SFN and LAN
It looked like ARI was in contention after 20210821 but the remaining intra-division games ruled them out.
We can afford to let TBA win 2 more games, and NYA 6
Let's say that TBA wins 2 of their remaining 3 with NYA
We can afford to let TBA win 0 more games, and BOS 8
Let's say that TBA wins 0 of their remaining 7 with BOS
We can afford to let TBA win 0 more games, and TOR 14
Let's say that TBA wins 0 of their remaining 6 with TOR
We can afford to let NYA win 5 more games, and BOS 1
Let's say that NYA wins 3 of their remaining 3 with BOS
We can afford to let NYA win 2 more games, and TOR 8
Let's say that NYA wins 2 of their remaining 7 with TOR
Teams eliminated from their division titles on 20210822: {'BAL'}
We can afford to let SFN win 2 more games, and LAN 3
Let's say that SFN wins 2 of their remaining 3 with LAN
We can afford to let SFN win 0 more games, and SDN 13
Let's say that SFN 

In [19]:
def wildcard_standings(standings_immutable, divisions, wildcard_count):
    contenders = set()
    if not wildcard_count:
        return pd.DataFrame()
    df = standings_immutable.copy()
    # I hate myself for this
    df['div'] = divisions['div']
    df['lg'] = divisions['lg']
    df['max_wins'] = 162 - df['L']


    df['first_place'] = False
    df.loc[df.groupby('div').head(1).index, 'first_place'] = True
    wildcard_wins_by_league = df.loc[df['first_place'] == False].sort_values(by=['W'], ascending=False).groupby('lg').nth(wildcard_count - 1)['W']
    def this_is_horrible(lg):
        return wildcard_wins_by_league.loc[lg]
    merge1 = df.merge(wildcard_wins_by_league, left_on=['lg'], right_index=True)
    return merge1.loc[merge1['max_wins'] >= merge1['W_y']][['W_x', 'L', 'div', 'lg', 'max_wins', 'first_place']].rename(columns={'W_x': 'W'})

current_standings = compute_standings(df.loc[df['completion_date'] <= '20210821'])
current_standings['div'] = DIVISIONS['div']
current_standings['lg'] = DIVISIONS['lg']
current_standings['max_wins'] = 162 - current_standings['L']

wildcard_standings(current_standings, DIVISIONS, wildcard_count=2)

,W,L,div,lg,max_wins,first_place
SFN,79,44,NLW,NL,118,True
LAN,78,46,NLW,NL,116,False
MIL,75,49,NLC,NL,113,True
SDN,68,57,NLW,NL,105,False
CIN,68,57,NLC,NL,105,False
ATL,67,56,NLE,NL,106,True
SLN,62,60,NLC,NL,102,False
PHI,62,61,NLE,NL,101,False
NYN,60,63,NLE,NL,99,False
COL,57,66,NLW,NL,96,False


In [20]:
def wildcard_contenders_naive(standings_immutable, divisions, wildcard_count):
    return set(wildcard_standings(standings_immutable, divisions, wildcard_count).index)
wildcard_contenders_naive(current_standings, DIVISIONS, wildcard_count=0)

set()

In [21]:
def wildcard_threats(standings_immutable, divisions, team, wildcard_count):
    # We only care about teams with more max_wins than us
    df = standings_immutable.copy()
    # this is so bad
    df['lg'] = divisions['lg']

    my_league = divisions.loc[team]['lg']
    my_max_wins = df.loc[team]['max_wins']
    df = df.loc[(df['lg'] == my_league) & (df['first_place'] == False)]
    df = df.drop(df.nlargest(wildcard_count - 1, columns=['W']).index)
    return my_max_wins - df['W']

wildcard_standings1 = wildcard_standings(current_standings, DIVISIONS, wildcard_count=2)
wildcard_threats(wildcard_standings1, DIVISIONS, 'TOR', wildcard_count=2)

OAK    35
BOS    35
SEA    39
TOR    41
ANA    43
CLE    45
DET    46
MIN    51
KCA    51
TEX    62
BAL    67
Name: W, dtype: int64

In [22]:
def games_between_wildcard_rivals_after_date(df, schedule, divisions, date_str, team, wildcard_count):
    remaining = all_matchups_after_date(df, schedule, date_str).groupby(['alpha1', 'alpha2'], as_index=False).size()
    standings = wildcard_standings(compute_standings(df.loc[(df['completion_date'] <= date_str)]), divisions, wildcard_count)
    threats = wildcard_threats(standings, divisions, team, wildcard_count)
    return remaining.loc[(remaining['alpha1'].isin(threats.index)) & (remaining['alpha2'].isin(threats.index))]

def is_wildcard_contender_with_rivalries(game_log, schedule, divisions, date_str, team, wildcard_count):
    matchups = games_between_wildcard_rivals_after_date(game_log, schedule, divisions, date_str, team, wildcard_count)
    standings = wildcard_standings(compute_standings(game_log.loc[(game_log['completion_date'] <= date_str)]), divisions, wildcard_count)
    # print(f'About to call wildcard_threats where standings has these columns: {standings.columns}')
    threats = wildcard_threats(standings, divisions, team, wildcard_count)
    sorted_rivals = sort_rivals(matchups, threats)
    return all_subset_sums(sorted_rivals, threats)


In [23]:
def count_teams(game_log, schedule, divisions):
    game_log_count = len(pd.unique(game_log[['home', 'visitor']].values.ravel('K')))
    schedule_count = len(pd.unique(schedule[['home', 'visitor']].values.ravel('K')))
    divisions_count = len(divisions.index)
    if (game_log_count != schedule_count or game_log_count != divisions_count):
        print(f'Teams in game log: {pd.unique(game_log[["home", "visitor"]].values.ravel("K"))} ')
        print(f'Teams in schedule: {pd.unique(schedule[["home", "visitor"]].values.ravel("K"))} ')
        print(f'Teams in division map: {divisions.index}')
    assert(game_log_count == schedule_count)
    assert(game_log_count == divisions_count)
    return game_log_count

count_teams(df, SCHEDULE, DIVISIONS)

30

In [24]:
def show_dumb_elimination_output3(df, schedule, divisions, wildcard_count=2):
    max_date = retro_to_datetime(df['completion_date'].max())
    min_date = retro_to_datetime(df['completion_date'].min())
    team_count = count_teams(df, schedule, divisions)
    games_per_season = get_season_length(schedule)
    print(f'This season has {team_count} teams and {games_per_season} for each team.')

    current_date = max_date
    div_contenders = set()
    tomorrows_div_contenders = None
    wildcard_contenders = set()
    tomorrows_wildcard_contenders = None
    tomorrows_contenders_any = None
    tomorrows_standings = None
    eliminations = {}
    while (
        current_date > min_date and        
        ((len(div_contenders) < team_count) or
         wildcard_count and len(wildcard_contenders) < team_count)):
        date_str = datetime_to_retro(current_date)
        print(f'Starting analysis of {date_str}')
        current_standings = compute_standings(df.loc[df['completion_date'] <= date_str])
        # print(current_standings)
        # MOVE THIS SHIT INTO SOMETHING EFFICIENT
        current_standings['div'] = divisions['div']
        current_standings['lg'] = divisions['lg']
        current_standings['max_wins'] = games_per_season - current_standings['L']
        # print(current_standings)
    
        div_contenders = division_contenders(current_standings, divisions, games_per_season)
        # print(f'naive division contenders: {div_contenders}')
        for supposed_contender in div_contenders.copy():
            # If you're in contention tomorrow, you're in contention today, so I am not going to
            # waste CPU time on you.
            if tomorrows_div_contenders and supposed_contender not in tomorrows_div_contenders:
                if not is_division_contender_with_rivalries(df, schedule, divisions, date_str, supposed_contender):
                    print(f'It looked like {supposed_contender} was in contention after {date_str} but the remaining intra-division games ruled them out.')
                    div_contenders.remove(supposed_contender)
        new_contenders = set()
        if tomorrows_div_contenders:
            new_contenders = div_contenders.difference(tomorrows_div_contenders)
        # if new_contenders:
            # print(f'Teams eliminated from their division titles on {datetime_to_retro(tomorrow)}: {new_contenders}')
        for eliminated_team in new_contenders:
            games_to_go_at_elimination = games_per_season - tomorrows_standings.loc[eliminated_team]['W'] - tomorrows_standings.loc[eliminated_team]['L']
            print(f'{eliminated_team} were eliminated from their division title on {datetime_to_retro(tomorrow)} with {games_to_go_at_elimination} games left to play.')
            new_pair = (datetime_to_retro(tomorrow), games_to_go_at_elimination)
            if eliminations.get(eliminated_team):
                eliminations[eliminated_team]['division'] = new_pair
            else:
                eliminations[eliminated_team] = {'division': new_pair}
        # print(f'PHI max wins: {current_standings.loc["PHI"]["max_wins"]}, SLN wins: {current_standings.loc["SLN"]["W"]}')
        # print(f'My busted view of the wildcard standings: {wildcard_standings(current_standings)}')
        wildcard_contenders = wildcard_contenders_naive(current_standings, divisions, wildcard_count)
        # print(f'naive wildcard contenders after {date_str} games: {wildcard_contenders}')
        for supposed_contender in wildcard_contenders.copy():
            # If you're in contention tomorrow, you're in contention today, so I am not going to
            # waste CPU time on you.
            if tomorrows_wildcard_contenders and supposed_contender not in tomorrows_wildcard_contenders:
                # print(f'let\'s see if {supposed_contender} is really still in wildcard contention')
                if not is_wildcard_contender_with_rivalries(df, schedule, divisions, date_str, supposed_contender, wildcard_count):
                    print(f'It looked like {supposed_contender} was in wildcard contention after {date_str} but the remaining intra-contender games ruled them out.')
                    wildcard_contenders.remove(supposed_contender)
        new_contenders = set()
        if tomorrows_wildcard_contenders:
            new_contenders = wildcard_contenders.difference(tomorrows_wildcard_contenders)
        for eliminated_team in new_contenders:
            games_to_go_at_elimination = games_per_season - tomorrows_standings.loc[eliminated_team]['W'] - tomorrows_standings.loc[eliminated_team]['L']
            print(f'{eliminated_team} were eliminated from wildcard contention on {datetime_to_retro(tomorrow)} with {games_to_go_at_elimination} games left to play.')
            new_pair = (datetime_to_retro(tomorrow), games_to_go_at_elimination)
            if eliminations.get(eliminated_team):
                eliminations[eliminated_team]['wildcard'] = new_pair
            else:
                eliminations[eliminated_team] = {'wildcard': new_pair}


        contenders_any = div_contenders.union(wildcard_contenders)
        if tomorrows_contenders_any:
            new_contenders = contenders_any.difference(tomorrows_contenders_any)
        if new_contenders:
            print(f'Teams eliminated from ALL postseason contention on {datetime_to_retro(tomorrow)}: {new_contenders}')
        tomorrow = current_date
        tomorrows_div_contenders = div_contenders.copy()
        tomorrows_wildcard_contenders = wildcard_contenders.copy()
        tomorrows_contenders_any = contenders_any.copy()
        tomorrows_standings = current_standings.copy()
        current_date = current_date - timedelta(days=1)
    return eliminations

#show_dumb_elimination_output3(df, SCHEDULE, DIVISIONS)

In [25]:
#gl2000 = load_game_log('./data/GL2000.TXT')
#sked2000 = load_schedule('./data/2000SKED.TXT')
#div2000 = divisions_for_year(NICKNAMES, 2000)

#show_dumb_elimination_output3(gl2000, sked2000, div2000, wildcard_count=1)

In [47]:
def wildcards_for_year(year):
    if year >= 2022:
        return 3
    if year == 2020:
        return "2020 was weird sorry"
    if year >= 2012:
        return 2
    if year >= 1995:
        return 1
    if year == "1994":
        return "1994 was weird sorry"
    return 0

def run_one_year(year):
    return show_dumb_elimination_output3(load_game_log(f'./data/GL{year}.TXT'), load_schedule(f'./data/{year}SKED.TXT'), divisions_for_year(NICKNAMES, year), wildcard_count=wildcards_for_year(year))

gl1950 = load_game_log(f'./data/GL1950.TXT')
sched1950 = load_schedule(f'./data/1950SKED.TXT')
pd.concat([gl1950['home'], gl1950['visitor']], axis = 0).value_counts()
gl1950.loc[gl1950['visitor'] == 'PHI']
compute_standings(gl1950)
# divisions_for_year(NICKNAMES, 1899)
run_one_year(1964)

,W,L
NYA,98,56
DET,95,59
BOS,94,60
CLE,92,62
PHI,91,63
BRO,89,65
NY1,86,68
BSN,83,71
SLN,78,75
WS1,67,87


In [50]:
# test case - the Mets after 1964-08-14
# and every date up to August 28 - they are obviously out on the 29th
print('can I execute anything at the top of this cell?')
mydict = {}
if mydict:
    print('an empty dict is true')
else:
    print('an empty dict is false')
    
def assign_wins_with_brute_force(sorted_rivals, threats_immutable, recursion_level=0):
    def myprint(mytext):
        if recursion_level <= 15:
            print (' ' * 2 * recursion_level, mytext)
    threats = threats_immutable.copy()
    if sorted_rivals.empty:
        return {}
    if all_subset_sums(sorted_rivals.rename(columns={'betterT': 'alpha1', 'worseT': 'alpha2'}), threats):
        pass
    else:
        return None

    row = sorted_rivals.iloc[0]
    remaining_rows = sorted_rivals.iloc[1: , :]
    games = row['size']
    betterT = row['betterT']
    worseT = row['worseT']
    threat1, threat2 = threats[betterT], threats[worseT]
    myprint(f'We can afford to let {betterT} win {threat1} more games, and {worseT} {threat2}')
    if threat1 < 0 or threat2 < 0:
        myprint(f'How the fuck did that get negative?')
        return None
    if games > threat1 + threat2:
        # No matter what, these games will lead to too many wins for one team.
        myprint(f'But there are {games} games left between {betterT} and {worseT}')
        return None
    minWinsForT1 = max(0, games - threat2)
    maxWinsForT1 = min(threat1, games)
    myprint(f'{betterT} has to win between {minWinsForT1} and {maxWinsForT1} of these {games} games.')
    for winsForT1 in range(minWinsForT1, maxWinsForT1+1):
        winsForT2 = games - winsForT1
        myprint(f'What if {betterT} won {winsForT1} out of {games} and {worseT} won {winsForT2}?')
        if winsForT1 > threat1:
            myprint(f'That is too many wins for {betterT} so we\'re done here.')
            return None
        if winsForT2 > threat2:
            myprint(f'That is too many wins for {worseT}.')
            continue
        # get ready to recurse
        new_threats = threats.copy()
        new_threats[betterT] -= winsForT1
        new_threats[worseT] -= winsForT2
        recurse = assign_wins_with_brute_force(remaining_rows, new_threats, recursion_level+1)
        if recurse != None:
            recurse[(betterT, worseT)] = winsForT1
            return recurse
        # If that recursive check was false, we'll keep going on the for loop.
    return None

def check_division_contention(date_str, year, team):
    game_log = load_game_log(f'./data/GL{year}.TXT')
    schedule = load_schedule(f'./data/{year}SKED.TXT')
    divisions = divisions_for_year(NICKNAMES, year)
    season_length = get_season_length(schedule)
    matchups = games_between_rivals_after_date(game_log, schedule, divisions, date_str, team)
    threats = divisional_threats(compute_standings(game_log.loc[(game_log['completion_date'] <= date_str)]), divisions, season_length, team)
    sum_mode = dumb_matrix_sum(matchups, threats)
    print(f'Sum mode says {sum_mode}')
    # sum mode errs on the side of True. If it's False, brute force will never return True.
    if not sum_mode:
        return False
    subset_mode = all_subset_sums(matchups, threats)
    print(f'Subset mode says {subset_mode}')
    # subset mode errs on the side of True. If it's False, brute force will never return True.
    if not subset_mode:
        return False
    sorted_rivals = sort_rivals(matchups, threats)
    easy_mode = is_division_contender_with_rivalries(game_log, schedule, divisions, date_str, team)
    # print(f'Easy mode says {easy_mode}')
    # Easy mode errs on the side of False. If it's true, brute force will never return False.
    # if easy_mode:
    #    return True
    brute_force = brute_force_rival_matchups(sorted_rivals, threats)
    print('brute force completed')
    return brute_force

def dumb_matrix_sum(matchups, threats):
    total_matchups = matchups['size'].sum()
    total_allowable_wins = threats.sum()
    return total_allowable_wins >= total_matchups

def powerset(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, s) if i & mask]

def all_subset_sums(matchups, threats):
    all_rivals = threats.index
    for subset in powerset(all_rivals):
        if len(subset) < 2:
            continue
        if 'betterT' in matchups.columns:
            team1, team2 = 'betterT', 'worseT'
        else:
            team1, team2 = 'alpha1', 'alpha2'
        subset_matchup_count = matchups.loc[(matchups[team1].isin(subset)) & (matchups[team2].isin(subset))]['size'].sum()
        allowable_win_count = threats.loc[threats.index.isin(subset)].sum()
        if subset_matchup_count <= allowable_win_count:
            # print(f'We can allow {subset} to win {allowable_win_count} and they only have {subset_matchup_count} games left so that seems okay.')
            pass
        else:
            # print(f'We can only allow {subset} to win {allowable_win_count} but they have {subset_matchup_count} games left.')
            return False
    return True

# they both say false for 19640823
# brute force says true for 19640814
# check_division_contention('20210821', 2021, 'BAL')


can I execute anything at the top of this cell?
an empty dict is false


In [51]:
for i in range(0,1):
    print(i)

0
